In [77]:
# Import Google Earth Engine
import ee
import geemap

In [78]:
# Initialize the Earth Engine module.
ee.Initialize()

In [79]:
# Coordinates for the bounds of a rectangle.
lon_min = -122.09;
lat_min = 37.42;
lon_max = -122.08;
lat_max = 37.43;


In [80]:
# Define an area of interest
aoi = ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])

In [94]:
# Hardcoded latitude and longitude 
lat, lon = 83.277, 17.7009;

In [95]:
# Removing cirrus pixel clouds and clouds from the map by Using the QA60 band to identify pixels affected by clouds (bit 10) and cirrus clouds (bit 11).
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [96]:
# Load the first image of Sentinel-2 Image Collection and filter by date and cloud cover
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-01-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)


In [97]:
# Set visualization parameters
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

In [98]:
# Create a map using geemap
Map = geemap.Map()

In [99]:
Map.set_center(lat, lon, 12)

In [100]:
Map.add_layer(dataset.mean(), visualization, 'RGB')

In [66]:
print(lat, lon)

6.9257 80.5688


In [67]:
# Calculate NDVI to help distinguish buildings from vegetation
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

In [101]:
# Display the map
Map

Map(center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [68]:
# Apply a threshold to detect buildings (NDVI is lower for non-vegetation areas)
buildings = ndvi.lt(0.2)  # Threshold NDVI to detect buildings

In [69]:
# Reduce to vectors (polygons representing building footprints)
building_vectors = buildings.reduceToVectors(geometryType='polygon', scale=10)

In [70]:
# Export to GeoJSON (optional)
export_task = ee.batch.Export.table.toDrive(collection=building_vectors, description='BuildingFootprints', fileFormat='GeoJSON')
export_task.start()

In [12]:
print('Image collection from a string:',
      ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').limit(3).getInfo())

img1 = ee.Image('COPERNICUS/S2_SR_HARMONIZED/20170328T083601_20170328T084228_T35RNK')
img2 = ee.Image('COPERNICUS/S2_SR_HARMONIZED/20170328T083601_20170328T084228_T35RNL')
img3 = ee.Image('COPERNICUS/S2_SR_HARMONIZED/20170328T083601_20170328T084228_T35RNM')
print('Image collection from a list of images:',
      ee.ImageCollection([img1, img2, img3]).getInfo())

print('Image collection from a single image:',
      ee.ImageCollection(img1).getInfo())

Image collection from a string: {'type': 'ImageCollection', 'bands': [], 'version': 1728224726998337, 'id': 'COPERNICUS/S2_SR_HARMONIZED', 'properties': {'date_range': [1490659200000, 1647907200000], 'period': 0, 'system:visualization_0_min': '0.0', 'type_name': 'ImageCollection', 'keywords': ['copernicus', 'esa', 'eu', 'msi', 'reflectance', 'sentinel', 'sr'], 'system:visualization_0_bands': 'B4,B3,B2', 'thumb': 'https://mw1.google.com/ges/dd/images/COPERNICUS_S2_SR_thumb.png', 'description': '<p>Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,\nincluding the monitoring of vegetation, soil and water cover,\nas well as observation of inland waterways and coastal areas.</p><p>The Sentinel-2 L2 data are downloaded from scihub. They were\ncomputed by running sen2cor. WARNING: ESA did not produce L2 data\nfor all L1 assets, and earlier L2 coverage is not global.</p><p>The assets contain\n12 UINT16 spectral bands repr